## Libraries


In [ ]:
# !pip install -r ~/code/benitomartin/FoodScore/requirements.txt

In [118]:
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers 
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications import VGG16,ResNet152
from tensorflow.keras.utils import load_img, img_to_array, to_categorical, image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator



import pickle

## Data import

In [2]:
coord = pd.DataFrame()

for i in range(1, 101, 1):
    path = f"../raw_data/UECFOOD100/{i}"
    data = pd.read_csv(f"{path}/bb_info.txt", sep=' ', header=0, index_col="img")
    data_df = pd.DataFrame(data)
    data_df["label"] = i
    coord = pd.concat([coord, data_df])
    


In [3]:
coord = coord.reset_index()

### DataFrame with label and coordinates

In [4]:
coord.head()

,img,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


In [5]:
coord = coord.rename(columns={"img": "img_name"})

In [6]:
coord.head()

,img_name,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


### Rescaling and Normalization

In [7]:
# function to normalize bounding box

def normalize_bbox(row):
    # Read in the image and get its dimensions
    image_path = f"../raw_data/UECFOOD100/{(row['label'])}/{(row['img_name'])}.jpg"
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Normalize the coordinates
    x1_norm = row['x1'] / width
    y1_norm = row['y1'] / height
    x2_norm = row['x2'] / width
    y2_norm = row['y2'] / height
    
    # Return normalized coordinates
    return pd.Series({'x1_norm': x1_norm, 'y1_norm': y1_norm, 'x2_norm': x2_norm, 'y2_norm': y2_norm})

# Apply the normalize_bbox function to each row of the DataFrame
normalized_bbox_df = coord.apply(normalize_bbox, axis=1)

# Concatenate the original DataFrame with the new normalized DataFrame
rescaled_coord = pd.concat([coord, normalized_bbox_df], axis=1).drop(columns=['x1', 'y1','x2','y2'])


In [8]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm
0,1,1,0.0000,0.238333,0.46250,0.810000
1,2,1,0.0250,0.346667,0.72750,0.931667
2,3,1,0.0025,0.183333,0.30375,0.683333
3,4,1,0.0000,0.395000,0.35750,0.893333
4,5,1,0.0100,0.046667,0.95125,0.975000


### add image paths

In [9]:
list_paths = [f"../raw_data/UECFOOD100/{int(row['label'])}/{int(row['img_name'])}.jpg" for _, row in coord.iterrows()]


In [10]:
rescaled_coord["paths"] = pd.DataFrame(list_paths).copy()

In [11]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.0000,0.238333,0.46250,0.810000,../raw_data/UECFOOD100/1/1.jpg
1,2,1,0.0250,0.346667,0.72750,0.931667,../raw_data/UECFOOD100/1/2.jpg
2,3,1,0.0025,0.183333,0.30375,0.683333,../raw_data/UECFOOD100/1/3.jpg
3,4,1,0.0000,0.395000,0.35750,0.893333,../raw_data/UECFOOD100/1/4.jpg
4,5,1,0.0100,0.046667,0.95125,0.975000,../raw_data/UECFOOD100/1/5.jpg


### balancing Dataset

In [12]:
coord.shape

(14611, 6)

In [13]:
def rebalancing(df: pd.DataFrame, classes: list, av_number: int = 110, random_state: int = 1) -> pd.DataFrame:
    df_new = df.copy()
    for class_ in classes:
        class_df = df_new[df_new['label'] == class_]
        class_count = len(class_df)
        if class_count > av_number:
            drop_indices = np.random.choice(class_df.index, class_count - av_number, replace=False)
            df_new = df_new.drop(drop_indices)
        else:
            pass
    return df_new

In [14]:
classes = list(set(rescaled_coord.label))

In [15]:
df = rebalancing(rescaled_coord, classes, av_number= 10, random_state=1)

In [16]:
rescaled_coord[rescaled_coord['label']==100].shape

(104, 7)

In [17]:
df[df['label']==100].shape

(10, 7)

### load downscaled pictures into array

In [18]:
from tqdm.auto import tqdm

/Users/katharinarolfs/.pyenv/versions/3.10.6/envs/FoodScore/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
df.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
16,20,1,0.000000,0.256667,0.571250,0.903333,../raw_data/UECFOOD100/1/20.jpg
49,64,1,0.000000,0.000000,1.000000,1.000000,../raw_data/UECFOOD100/1/64.jpg
127,7077,1,0.071667,0.407008,0.496667,1.000000,../raw_data/UECFOOD100/1/7077.jpg
146,9222,1,0.078652,0.212500,0.932584,0.731250,../raw_data/UECFOOD100/1/9222.jpg
240,13233,1,0.003125,0.566667,0.343750,1.000000,../raw_data/UECFOOD100/1/13233.jpg


In [20]:
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
8283,5304,53,0.456250,0.433333,0.787500,0.958333,../raw_data/UECFOOD100/53/5304.jpg
11032,7932,74,0.000000,0.000000,1.000000,1.000000,../raw_data/UECFOOD100/74/7932.jpg
11052,7944,75,0.000000,0.000000,0.920968,1.000000,../raw_data/UECFOOD100/75/7944.jpg
9926,31,67,0.096859,0.183267,0.403141,0.378486,../raw_data/UECFOOD100/67/31.jpg
6909,4094,42,0.011765,0.051370,0.970588,0.900685,../raw_data/UECFOOD100/42/4094.jpg


In [21]:
df_shuffled.to_csv('df_shuffled.csv')

In [22]:
color_order = "BGR"
dims = (224,224)

images = np.empty((len(df_shuffled), dims[0], dims[1], 3), dtype=np.float32)

for i, path in enumerate(tqdm(df_shuffled.paths.values)):
    img = cv2.imread(path)
    img = cv2.resize(img, dims, interpolation=cv2.INTER_AREA)
    if color_order == "RGB":
        img = img[:,:,::-1]
    images[i, :, :, :] = img/255

100%|██████████████████████████████████████| 1000/1000 [00:02<00:00, 435.09it/s]


In [23]:
labels = np.array(df_shuffled.label)
bboxes = np.array(df_shuffled[['x1_norm','y1_norm','x2_norm','y2_norm']], dtype="float32")
paths = np.array(df_shuffled.paths)

In [24]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [25]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

In [26]:
len(set(df_shuffled.label))

100

In [89]:
tvImages, testImages,tvLabels, testLabels =\
train_test_split(images,
                 labels,
                 test_size=0.20,
                 random_state=42)

In [90]:
trainImages, valImages,trainLabels, valLabels=\
train_test_split(tvImages,
                 tvLabels,
                 test_size=0.20,
                 random_state=42)

In [91]:
type(trainImages)

numpy.ndarray

## Model

In [125]:
import tensorflow as tf

In [124]:
tf.keras.layers.Ra

NameError: name 'tf' is not defined

In [122]:
# Define the data augmentation transformations
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5],
    channel_shift_range=50,
    )


# Fit the datagen to your train data
datagen.fit(trainImages)

# Generate augmented data using the datagen generator
train_data_augmented = datagen.flow(trainImages, batch_size=32)

# Create a symbolic tensor for the input
input_tensor = layers.Input(shape=(224, 224, 3))

# Load pre-trained ResNet152 model with the input tensor
base_model = ResNet152(
    include_top=False,
    weights='imagenet',
    input_tensor=input_tensor
)

# Freeze layers in base model
layers.trainable = False

# Region Proposal Network
rpn = layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding="same", activation="gelu")(base_model.output)
rpn_class = layers.Conv2D(filters=2, kernel_size=(1,1), activation="softmax", name="rpn_class")(rpn)

# Classification Head
flatten = layers.GlobalAveragePooling2D()(base_model.output)
flatten = layers.Flatten()(flatten)

softmax_head = layers.Dense(128, activation="gelu", kernel_regularizer=l2(0.02))(flatten)
softmax_head = layers.Dropout(0.5)(softmax_head)
softmax_head = layers.Dense(64, activation="gelu", kernel_regularizer=l2(0.04))(softmax_head)
softmax_head = layers.Dropout(0.5)(softmax_head)
softmax_head = layers.Dense(len(set(df_shuffled.label)), activation="softmax", name="class_label", kernel_regularizer=l2(0.01))(softmax_head)

# Combine the model heads
outputs = [softmax_head]
model = Model(inputs=input_tensor, outputs=outputs)


In [121]:
losses = {
    "class_label": 'categorical_crossentropy',}

In [95]:
lossWeights = {
    "class_label": 1.0,}

In [96]:
trainTargets = {
    "class_label": trainLabels,}

In [97]:
testTargets = {
    "class_label": testLabels,}

In [98]:
valTargets = {
    "class_label": valLabels,}

In [99]:
metrics = {
    "class_label": "categorical_accuracy",}

In [100]:
opt = Adam(0.001)


model.compile(loss=losses, 
              optimizer=opt, 
              metrics=metrics, 
              loss_weights=lossWeights)

print(model.summary())

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block6_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block6_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block6_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_3_conv (Conv2D)   (None, 28, 28, 512)  66048       ['conv3_block6_2_relu[0][0]']    
                                                                                                  
 conv3_block6_3_bn (BatchNormal  (None, 28, 28, 512)  2048       ['conv3_block6_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block1_0_conv (Conv2D)   (None, 14, 14, 1024  525312      ['conv3_block8_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block1_3_conv (Conv2D)   (None, 14, 14, 1024  263168      ['conv4_block1_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block1_0_bn (BatchNormal  (None, 14, 14, 1024  4096       ['conv4_block1_0_conv[0][0]']    
 ization)                       )                                                                 
          

...

                                                                                                  
 conv4_block28_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block27_out[0][0]']      
                                                                                                  
 conv4_block28_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block28_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block28_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block28_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block28_2_conv (Conv2D)  (None, 14, 14, 256)  590080      ['conv4_block28_1_relu[0][0]']   
          

                                )                                                                 
                                                                                                  
 conv4_block31_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block30_out[0][0]']      
                                                                                                  
 conv4_block31_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block31_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block31_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block31_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_blo

 conv4_block33_out (Activation)  (None, 14, 14, 1024  0          ['conv4_block33_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv4_block34_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block33_out[0][0]']      
                                                                                                  
 conv4_block34_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block34_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block34_1_bn[0][0]']     
 on)                                                                                              
          

                                                                                                  
 conv4_block36_out (Activation)  (None, 14, 14, 1024  0          ['conv4_block36_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv5_block1_1_conv (Conv2D)   (None, 7, 7, 512)    524800      ['conv4_block36_out[0][0]']      
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)       

                                                                  'conv5_block3_3_bn[0][0]']      
                                                                                                  
 conv5_block3_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block3_add[0][0]']       
                                                                                                  
 global_average_pooling2d_5 (Gl  (None, 2048)        0           ['conv5_block3_out[0][0]']       
 obalAveragePooling2D)                                                                            
                                                                                                  
 flatten_5 (Flatten)            (None, 2048)         0           ['global_average_pooling2d_5[0][0
                                                                 ]']                              
                                                                                                  
 dense_10 

In [101]:
es = EarlyStopping(monitor = 'categorical_accuracy',
                   patience = 20,
                   verbose = 0,
                   restore_best_weights = True)

In [112]:
modelCheckpoint = ModelCheckpoint("{}.h5".format(model), 
                                  monitor="val_loss", 
                                  verbose=0, 
                                  save_best_only=True)
LRreducer = ReduceLROnPlateau(monitor="val_loss", 
                              factor = 0.1, 
                              patience=3, 
                              verbose=1, 
                              min_lr=0)

In [113]:
history_100_label_only = model.fit(
    trainImages,
    trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=32,
    epochs=1000,
    verbose=1,
    use_multiprocessing=True,
    workers=8,
    callbacks = [es, modelCheckpoint,LRreducer],
    )

Epoch 1/1000
20/20 [==============================] - 70s 4s/step - loss: 4.5111 - categorical_accuracy: 0.0234 - val_loss: 4.9079 - val_categorical_accuracy: 0.0125 - lr: 0.0010
Epoch 2/1000
20/20 [==============================] - 80s 4s/step - loss: 4.4657 - categorical_accuracy: 0.0359 - val_loss: 5.3174 - val_categorical_accuracy: 0.0125 - lr: 0.0010
Epoch 3/1000
20/20 [==============================] - 122s 6s/step - loss: 4.4706 - categorical_accuracy: 0.0312 - val_loss: 5.0949 - val_categorical_accuracy: 0.0125 - lr: 0.0010
Epoch 4/1000
20/20 [==============================] - ETA: 0s - loss: 4.4483 - categorical_accuracy: 0.0312
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
20/20 [==============================] - 108s 5s/step - loss: 4.4483 - categorical_accuracy: 0.0312 - val_loss: 5.7965 - val_categorical_accuracy: 0.0063 - lr: 0.0010
Epoch 5/1000
20/20 [==============================] - 107s 5s/step - loss: 4.4116 - categorical_accuracy: 0.02

## Save model

In [ ]:
# Resnet152
models.save(model)

In [ ]:
def plot_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('accuracy')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()